In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7345,2021-02-07T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1198,178,1376,37633,...,NaN,12.0,NaN,NaN,141554.0,0.0,1571253.0,390266.0,ITG,ITG1
7346,2021-02-07T17:00:00,ITA,9,Toscana,43.769231,11.255889,650,108,758,9682,...,NaN,7.0,NaN,NaN,138443.0,175.0,2216022.0,123937.0,ITI,ITI1
7347,2021-02-07T17:00:00,ITA,10,Umbria,43.106758,12.388247,411,73,484,6300,...,"Si fa presente che 38 dei ricoveri NON UTI, so...",5.0,NaN,NaN,38484.0,0.0,620732.0,59021.0,ITI,ITI2
7348,2021-02-07T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,24,3,27,135,...,NaN,0.0,NaN,NaN,7843.0,0.0,70385.0,1437.0,ITC,ITC2


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note', 'note_test', 'note_casi', 'codice_nuts_1', 'codice_nuts_2']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7345,2021-02-07T17:00:00,19,Sicilia,1198,178,1376,37633,39009,-257,574,...,NaN,NaN,141554,1961519,1024409.0,12.0,141554.0,0.0,1571253.0,390266.0
7346,2021-02-07T17:00:00,9,Toscana,650,108,758,9682,10440,175,668,...,NaN,NaN,138618,2339959,1243732.0,7.0,138443.0,175.0,2216022.0,123937.0
7347,2021-02-07T17:00:00,10,Umbria,411,73,484,6300,6784,202,382,...,NaN,NaN,38484,679753,292454.0,5.0,38484.0,0.0,620732.0,59021.0
7348,2021-02-07T17:00:00,2,Valle d'Aosta,24,3,27,135,162,-6,1,...,NaN,NaN,7843,71822,42029.0,0.0,7843.0,0.0,70385.0,1437.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-02-07', '2021-02-06')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-07T17:00:00,1,Piemonte,2017,140,2157,10165,12322,140,624,...,NaN,NaN,228279,2186106,1132152.0,6.0,225651.0,2628.0,1924723.0,261383.0
1,2021-02-07T17:00:00,2,Valle d'Aosta,24,3,27,135,162,-6,1,...,NaN,NaN,7843,71822,42029.0,0.0,7843.0,0.0,70385.0,1437.0
2,2021-02-07T17:00:00,3,Lombardia,3467,358,3825,45015,48840,531,1515,...,NaN,NaN,549485,5879680,2894441.0,14.0,547114.0,2371.0,5687920.0,191760.0
3,2021-02-07T17:00:00,5,Veneto,1110,153,1263,25707,26970,-247,496,...,NaN,NaN,316836,4366107,1340565.0,3.0,314988.0,1848.0,3822282.0,543825.0
4,2021-02-07T17:00:00,6,Friuli Venezia Giulia,521,63,584,10203,10787,-85,285,...,NaN,NaN,70112,1196172,433968.0,3.0,66912.0,3200.0,1145376.0,50796.0
5,2021-02-07T17:00:00,7,Liguria,608,61,669,3982,4651,57,364,...,NaN,NaN,71749,902861,389514.0,3.0,71749.0,0.0,846189.0,56672.0
6,2021-02-07T17:00:00,8,Emilia-Romagna,1950,181,2131,41130,43261,613,1382,...,NaN,NaN,226926,3264051,1390037.0,7.0,226859.0,67.0,3080571.0,183480.0
7,2021-02-07T17:00:00,9,Toscana,650,108,758,9682,10440,175,668,...,NaN,NaN,138618,2339959,1243732.0,7.0,138443.0,175.0,2216022.0,123937.0
8,2021-02-07T17:00:00,10,Umbria,411,73,484,6300,6784,202,382,...,NaN,NaN,38484,679753,292454.0,5.0,38484.0,0.0,620732.0,59021.0
9,2021-02-07T17:00:00,11,Marche,528,74,602,7024,7626,1,394,...,NaN,NaN,58141,722696,435964.0,6.0,58141.0,0.0,688122.0,34574.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-06T17:00:00,1,Piemonte,2007,140,2147,10035,12182,-13,717,...,NaN,NaN,227655,2178038,1128845.0,8.0,225079.0,2576.0,1920280.0,257758.0
1,2021-02-06T17:00:00,2,Valle d'Aosta,24,3,27,141,168,-11,8,...,NaN,NaN,7842,71659,41965.0,0.0,7842.0,0.0,70246.0,1413.0
2,2021-02-06T17:00:00,3,Lombardia,3540,354,3894,44415,48309,107,1923,...,NaN,NaN,547970,5852056,2885842.0,22.0,545694.0,2276.0,5665314.0,186742.0
3,2021-02-06T17:00:00,5,Veneto,1123,158,1281,25936,27217,-709,831,...,NaN,NaN,316340,4348682,1338389.0,9.0,314492.0,1848.0,3813994.0,534688.0
4,2021-02-06T17:00:00,6,Friuli Venezia Giulia,533,63,596,10276,10872,-183,413,...,NaN,NaN,69827,1191476,432116.0,4.0,66691.0,3136.0,1141573.0,49903.0
5,2021-02-06T17:00:00,7,Liguria,609,61,670,3924,4594,107,276,...,NaN,NaN,71385,898257,388159.0,3.0,71385.0,0.0,843214.0,55043.0
6,2021-02-06T17:00:00,8,Emilia-Romagna,1956,183,2139,40509,42648,-488,1383,...,NaN,NaN,225545,3247836,1385467.0,17.0,225479.0,66.0,3070594.0,177242.0
7,2021-02-06T17:00:00,9,Toscana,643,109,752,9513,10265,173,708,...,NaN,NaN,137950,2327331,1237112.0,10.0,137785.0,165.0,2206468.0,120863.0
8,2021-02-06T17:00:00,10,Umbria,388,70,458,6124,6582,184,351,...,NaN,NaN,38102,671839,291099.0,6.0,38102.0,0.0,616821.0,55018.0
9,2021-02-06T17:00:00,11,Marche,531,75,606,7019,7625,-302,436,...,NaN,NaN,57747,718208,433086.0,4.0,57747.0,0.0,684797.0,33411.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2021-02-07T17:00:00,1,Piemonte,2017,140,2157,10165,12322,140,624,...,NaN,NaN,228279,2186106,1132152.0,6.0,225651.0,2628.0,1924723.0,261383.0
1,2021-02-07T17:00:00,2,Valle d'Aosta,24,3,27,135,162,-6,1,...,NaN,NaN,7843,71822,42029.0,0.0,7843.0,0.0,70385.0,1437.0
2,2021-02-07T17:00:00,3,Lombardia,3467,358,3825,45015,48840,531,1515,...,NaN,NaN,549485,5879680,2894441.0,14.0,547114.0,2371.0,5687920.0,191760.0
3,2021-02-07T17:00:00,4,Trentino-Alto Adige,378,61,439,7526,7965,349,645,...,0.0,0.0,73389,1097610,349672.0,1.0,63881.0,9508.0,965931.0,131679.0
4,2021-02-07T17:00:00,5,Veneto,1110,153,1263,25707,26970,-247,496,...,NaN,NaN,316836,4366107,1340565.0,3.0,314988.0,1848.0,3822282.0,543825.0
5,2021-02-07T17:00:00,6,Friuli Venezia Giulia,521,63,584,10203,10787,-85,285,...,NaN,NaN,70112,1196172,433968.0,3.0,66912.0,3200.0,1145376.0,50796.0
6,2021-02-07T17:00:00,7,Liguria,608,61,669,3982,4651,57,364,...,NaN,NaN,71749,902861,389514.0,3.0,71749.0,0.0,846189.0,56672.0
7,2021-02-07T17:00:00,8,Emilia-Romagna,1950,181,2131,41130,43261,613,1382,...,NaN,NaN,226926,3264051,1390037.0,7.0,226859.0,67.0,3080571.0,183480.0
8,2021-02-07T17:00:00,9,Toscana,650,108,758,9682,10440,175,668,...,NaN,NaN,138618,2339959,1243732.0,7.0,138443.0,175.0,2216022.0,123937.0
9,2021-02-07T17:00:00,10,Umbria,411,73,484,6300,6784,202,382,...,NaN,NaN,38484,679753,292454.0,5.0,38484.0,0.0,620732.0,59021.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,0,2021-02-06T17:00:00,1,Piemonte,2007,140,2147,10035,12182,-13,...,NaN,NaN,227655,2178038,1128845.0,8.0,225079.0,2576.0,1920280.0,257758.0
1,1,2021-02-06T17:00:00,2,Valle d'Aosta,24,3,27,141,168,-11,...,NaN,NaN,7842,71659,41965.0,0.0,7842.0,0.0,70246.0,1413.0
2,2,2021-02-06T17:00:00,3,Lombardia,3540,354,3894,44415,48309,107,...,NaN,NaN,547970,5852056,2885842.0,22.0,545694.0,2276.0,5665314.0,186742.0
3,19,2021-02-06T17:00:00,4,Trentino-Alto Adige,373,62,435,7181,7616,-1272,...,0.0,0.0,72744,1089932,348671.0,1.0,63526.0,9218.0,962306.0,127626.0
4,3,2021-02-06T17:00:00,5,Veneto,1123,158,1281,25936,27217,-709,...,NaN,NaN,316340,4348682,1338389.0,9.0,314492.0,1848.0,3813994.0,534688.0
5,4,2021-02-06T17:00:00,6,Friuli Venezia Giulia,533,63,596,10276,10872,-183,...,NaN,NaN,69827,1191476,432116.0,4.0,66691.0,3136.0,1141573.0,49903.0
6,5,2021-02-06T17:00:00,7,Liguria,609,61,670,3924,4594,107,...,NaN,NaN,71385,898257,388159.0,3.0,71385.0,0.0,843214.0,55043.0
7,6,2021-02-06T17:00:00,8,Emilia-Romagna,1956,183,2139,40509,42648,-488,...,NaN,NaN,225545,3247836,1385467.0,17.0,225479.0,66.0,3070594.0,177242.0
8,7,2021-02-06T17:00:00,9,Toscana,643,109,752,9513,10265,173,...,NaN,NaN,137950,2327331,1237112.0,10.0,137785.0,165.0,2206468.0,120863.0
9,8,2021-02-06T17:00:00,10,Umbria,388,70,458,6124,6582,184,...,NaN,NaN,38102,671839,291099.0,6.0,38102.0,0.0,616821.0,55018.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione
0,2021-02-07T17:00:00,1,Piemonte,2017,140,2157,10165,12322,140,624,...,NaN,228279,2186106,1132152.0,6.0,225651.0,2628.0,1924723.0,261383.0,4341375
1,2021-02-07T17:00:00,2,Valle d'Aosta,24,3,27,135,162,-6,1,...,NaN,7843,71822,42029.0,0.0,7843.0,0.0,70385.0,1437.0,125501
2,2021-02-07T17:00:00,3,Lombardia,3467,358,3825,45015,48840,531,1515,...,NaN,549485,5879680,2894441.0,14.0,547114.0,2371.0,5687920.0,191760.0,10103969
3,2021-02-07T17:00:00,4,Trentino-Alto Adige,378,61,439,7526,7965,349,645,...,0.0,73389,1097610,349672.0,1.0,63881.0,9508.0,965931.0,131679.0,1074819
4,2021-02-07T17:00:00,5,Veneto,1110,153,1263,25707,26970,-247,496,...,NaN,316836,4366107,1340565.0,3.0,314988.0,1848.0,3822282.0,543825.0,4907704
5,2021-02-07T17:00:00,6,Friuli Venezia Giulia,521,63,584,10203,10787,-85,285,...,NaN,70112,1196172,433968.0,3.0,66912.0,3200.0,1145376.0,50796.0,1211357
6,2021-02-07T17:00:00,7,Liguria,608,61,669,3982,4651,57,364,...,NaN,71749,902861,389514.0,3.0,71749.0,0.0,846189.0,56672.0,1543127
7,2021-02-07T17:00:00,8,Emilia-Romagna,1950,181,2131,41130,43261,613,1382,...,NaN,226926,3264051,1390037.0,7.0,226859.0,67.0,3080571.0,183480.0,4467118
8,2021-02-07T17:00:00,9,Toscana,650,108,758,9682,10440,175,668,...,NaN,138618,2339959,1243732.0,7.0,138443.0,175.0,2216022.0,123937.0,3722729
9,2021-02-07T17:00:00,10,Umbria,411,73,484,6300,6784,202,382,...,NaN,38484,679753,292454.0,5.0,38484.0,0.0,620732.0,59021.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,popolazione,totale_posti_terapia_intensiva
0,2021-02-07T17:00:00,1,Piemonte,2017,140,2157,10165,12322,140,624,...,228279,2186106,1132152.0,6.0,225651.0,2628.0,1924723.0,261383.0,4341375,575
1,2021-02-07T17:00:00,2,Valle d'Aosta,24,3,27,135,162,-6,1,...,7843,71822,42029.0,0.0,7843.0,0.0,70385.0,1437.0,125501,20
2,2021-02-07T17:00:00,3,Lombardia,3467,358,3825,45015,48840,531,1515,...,549485,5879680,2894441.0,14.0,547114.0,2371.0,5687920.0,191760.0,10103969,1036
3,2021-02-07T17:00:00,4,Trentino-Alto Adige,378,61,439,7526,7965,349,645,...,73389,1097610,349672.0,1.0,63881.0,9508.0,965931.0,131679.0,1074819,106
4,2021-02-07T17:00:00,5,Veneto,1110,153,1263,25707,26970,-247,496,...,316836,4366107,1340565.0,3.0,314988.0,1848.0,3822282.0,543825.0,4907704,1016
5,2021-02-07T17:00:00,6,Friuli Venezia Giulia,521,63,584,10203,10787,-85,285,...,70112,1196172,433968.0,3.0,66912.0,3200.0,1145376.0,50796.0,1211357,180
6,2021-02-07T17:00:00,7,Liguria,608,61,669,3982,4651,57,364,...,71749,902861,389514.0,3.0,71749.0,0.0,846189.0,56672.0,1543127,209
7,2021-02-07T17:00:00,8,Emilia-Romagna,1950,181,2131,41130,43261,613,1382,...,226926,3264051,1390037.0,7.0,226859.0,67.0,3080571.0,183480.0,4467118,634
8,2021-02-07T17:00:00,9,Toscana,650,108,758,9682,10440,175,668,...,138618,2339959,1243732.0,7.0,138443.0,175.0,2216022.0,123937.0,3722729,523
9,2021-02-07T17:00:00,10,Umbria,411,73,484,6300,6784,202,382,...,38484,679753,292454.0,5.0,38484.0,0.0,620732.0,59021.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-02-07T17:00:00,15,Campania,1500,105,1605,63867,65472,1232,1741,...,20.79,1234,195,19581,7455.0,3,506,8.89,0.03009,4.01
1,2021-02-07T17:00:00,3,Lombardia,3467,358,3825,45015,48840,531,1515,...,34.56,600,-408,27624,8599.0,58,926,5.48,0.01499,5.44
2,2021-02-07T17:00:00,8,Emilia-Romagna,1950,181,2131,41130,43261,613,1382,...,28.55,621,-1,16215,4570.0,25,743,8.52,0.03094,5.08
3,2021-02-07T17:00:00,12,Lazio,2232,264,2496,50862,53358,-3085,920,...,31.17,-3038,-94,22166,14591.0,20,3985,4.15,0.01568,3.62
4,2021-02-07T17:00:00,16,Puglia,1434,162,1596,49670,51266,191,765,...,43.90,179,-161,8751,1681.0,8,566,8.74,0.01909,3.22
5,2021-02-07T17:00:00,9,Toscana,650,108,758,9682,10440,175,668,...,20.65,169,-40,12628,6620.0,22,471,5.29,0.01794,3.72
6,2021-02-07T17:00:00,4,Trentino-Alto Adige,378,61,439,7526,7965,349,645,...,57.55,345,-435,7678,1001.0,10,286,8.40,0.06001,6.83
7,2021-02-07T17:00:00,1,Piemonte,2017,140,2157,10165,12322,140,624,...,24.35,130,-93,8068,3307.0,8,476,7.73,0.01437,5.26
8,2021-02-07T17:00:00,19,Sicilia,1198,178,1376,37633,39009,-257,574,...,30.27,-228,-262,24633,4431.0,25,806,2.33,0.01155,2.85
9,2021-02-07T17:00:00,5,Veneto,1110,153,1263,25707,26970,-247,496,...,15.06,-229,-335,17425,2176.0,28,715,2.85,0.01011,6.46


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,73,44,117,62.39
1,Trentino-Alto Adige,61,45,106,57.55
2,Marche,74,69,143,51.75
3,Puglia,162,207,369,43.90
4,Friuli Venezia Giulia,63,117,180,35.00
5,Lombardia,358,678,1036,34.56
6,Abruzzo,51,97,148,34.46
7,Molise,11,23,34,32.35
8,Lazio,264,583,847,31.17
9,Sicilia,178,410,588,30.27


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,...,NaN,NaN,0,3,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2021-02-05T17:00:00,22,P.A. Trento,142,32,174,2341,2515,73,202,...,NaN,NaN,28455,542779,163001.0,0.0,25690.0,2765.0,519349.0,23430.0
696,2021-02-06T17:00:00,21,P.A. Bolzano,242,34,276,4716,4992,-1381,802,...,NaN,NaN,44011,543822,185338.0,0.0,37780.0,6231.0,441234.0,102588.0
697,2021-02-06T17:00:00,22,P.A. Trento,131,28,159,2465,2624,109,278,...,NaN,NaN,28733,546110,163333.0,1.0,25746.0,2987.0,521072.0,25038.0
698,2021-02-07T17:00:00,21,P.A. Bolzano,242,35,277,5012,5289,297,470,...,NaN,NaN,44481,549137,186072.0,1.0,38083.0,6398.0,443564.0,105573.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2021-02-03T17:00:00,4,Trentino-Alto Adige,399,65,464,7781,8245,841,914,...,8.0,12142.0,1144.0,852.0,14.01,5.63,7.53,79.90,-4.41,11.36
346,2021-02-04T17:00:00,4,Trentino-Alto Adige,393,67,460,7873,8333,88,987,...,7.0,11523.0,1086.0,92.0,14.57,5.52,8.57,90.88,3.08,1.07
347,2021-02-05T17:00:00,4,Trentino-Alto Adige,397,67,464,8424,8888,555,889,...,6.0,11594.0,1106.0,551.0,14.44,5.22,7.67,80.38,0.00,6.66
348,2021-02-06T17:00:00,4,Trentino-Alto Adige,373,62,435,7181,7616,-1272,1080,...,6.0,12681.0,1118.0,-1243.0,14.25,5.71,8.52,96.60,-7.46,-14.31


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
